In [ ]:
from vitalDSP.utils.synthesize_data import generate_ecg_signal
import numpy as np
from plotly import graph_objects as go
# import plotly.io as pio
# pio.renderers.default = "sphinx_gallery"
# pio.renderers.default = "jupyterlab"  # or "plotly_mimetype"
# from IPython.display import display, HTML
# display(HTML('<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js"></script>'))
# import matplotlib.pyplot as plt
from vitalDSP.utils.peak_detection import PeakDetection
from vitalDSP.physiological_features.waveform import WaveformMorphology

sfecg = 128
N = 15
Anoise = 0.05
hrmean = 70
ecg_signal = generate_ecg_signal(
    sfecg=sfecg, N=N, Anoise=Anoise, hrmean=hrmean
)

detector = PeakDetection(
    ecg_signal,"ecg_r_peak", **{
        "distance": 50, 
        "window_size": 7, 
        "threshold_factor":1.6, 
        "search_window":6}
    )

# rpeaks = detector.detect_peaks()

waveform = WaveformMorphology(ecg_signal, fs=128, signal_type="ECG")
rpeaks = waveform.r_peaks
q_valleys = waveform.detect_q_valley()
p_peaks = waveform.detect_p_peak()
s_valleys = waveform.detect_s_valley()
t_peaks = waveform.detect_t_peak()
    
fig = go.Figure()
    # Plot the ECG signal
fig.add_trace(go.Scatter(x=np.arange(len(ecg_signal)), y=ecg_signal, mode="lines", name="ECG Signal"))

# Plot R-peaks
fig.add_trace(go.Scatter(x=rpeaks, y=ecg_signal[rpeaks], mode="markers", name="R Peaks", marker=dict(color="red", size=8)))
fig.add_trace(go.Scatter(x=q_valleys, y=ecg_signal[q_valleys], mode="markers", name="Q Valleys", marker=dict(color="green", size=8)))
fig.add_trace(go.Scatter(x=s_valleys, y=ecg_signal[s_valleys], mode="markers", name="S Valleys", marker=dict(size=8)))
fig.add_trace(go.Scatter(x=p_peaks, y=ecg_signal[p_peaks], mode="markers", name="P Peaks", marker=dict(size=8)))
fig.add_trace(go.Scatter(x=t_peaks, y=ecg_signal[t_peaks], mode="markers", name="T Peaks", marker=dict(size=8)))
fig.update_layout(
        title="ECG Signal with QRS-peaks/valleys and P, T peaks",
        xaxis_title="Samples",
        yaxis_title="Amplitude",
        showlegend=True
)
fig.show()

In [ ]:
import numpy as np
import vitalDSP
from vitalDSP.utils.synthesize_data import generate_ecg_signal, generate_synthetic_ppg
from vitalDSP.filtering.signal_filtering import SignalFiltering
from vitalDSP.physiological_features.waveform import WaveformMorphology
from vitalDSP.physiological_features.time_domain import TimeDomainFeatures
from vitalDSP.physiological_features.frequency_domain import FrequencyDomainFeatures
from vitalDSP.physiological_features.hrv_analysis import HRVFeatures
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Generate synthetic ECG signal using VitalDSP
fs = 128  # Sampling frequency
duration = 10  # seconds
hr_mean = 75  # beats per minute
noise_amplitude = 0.1

print("Generating synthetic ECG signal using VitalDSP...")
ecg_signal = generate_ecg_signal(
    sfecg=fs, 
    duration=duration, 
    Anoise=noise_amplitude, 
    hrmean=hr_mean
)

print(f"Generated ECG signal: {len(ecg_signal)} samples at {fs} Hz")

# Also generate a synthetic PPG signal for comparison
print("\nGenerating synthetic PPG signal using VitalDSP...")
ppg_time, ppg_signal = generate_synthetic_ppg(
    duration=duration,
    sampling_rate=fs,
    heart_rate=hr_mean,
    noise_level=noise_amplitude
)

print(f"Generated PPG signal: {len(ppg_signal)} samples at {fs} Hz")

# Initialize signal filtering
sf = SignalFiltering(ecg_signal)

# Apply bandpass filter to remove noise and artifacts
filtered_signal = sf.bandpass(lowcut=0.3, highcut=20.0, fs=fs)
print("Applied bandpass filter (0.3-20 Hz)")

# Extract comprehensive features using WaveformMorphology
print("Extracting comprehensive features using WaveformMorphology...")
wm = WaveformMorphology(filtered_signal, fs=fs, signal_type="ECG")

# Extract all available features
try:
    # Get basic waveform features
    r_peaks = wm.r_peaks
    q_valleys = wm.detect_q_valley()
    s_valleys = wm.detect_s_valley()
    p_peaks = wm.detect_p_peak()
    t_peaks = wm.detect_t_peak()
    
    print(f"Detected {len(r_peaks)} R-peaks")
    print(f"Detected {len(q_valleys)} Q-valleys")
    print(f"Detected {len(s_valleys)} S-valleys")
    print(f"Detected {len(p_peaks)} P-peaks")
    print(f"Detected {len(t_peaks)} T-peaks")
    
    # Calculate heart rate
    if len(r_peaks) > 1:
        rr_intervals = np.diff(r_peaks) / fs * 1000  # Convert to milliseconds
        heart_rate = 60000 / np.mean(rr_intervals)  # BPM
        print(f"Average heart rate: {heart_rate:.1f} BPM")
        
        # Extract HRV features using TimeDomainFeatures
        print("Extracting HRV features...")
        hrv_features = TimeDomainFeatures(rr_intervals)
        
        hrv_results = {
            'SDNN': hrv_features.compute_sdnn(),
            'RMSSD': hrv_features.compute_rmssd(),
            'NN50': hrv_features.compute_nn50(),
            'pNN50': hrv_features.compute_pnn50(),
            'Mean_NN': hrv_features.compute_mean_nn(),
            'Median_NN': hrv_features.compute_median_nn(),
            'IQR_NN': hrv_features.compute_iqr_nn(),
            'CVNN': hrv_features.compute_cvnn(),
            'HRV_Triangular_Index': hrv_features.compute_hrv_triangular_index(),
            'TINN': hrv_features.compute_tinn()
        }
        
        print("HRV Features:")
        for feature, value in hrv_results.items():
            print(f"  {feature}: {value:.4f}")
    
    # Extract morphological features
    print("Extracting morphological features...")
    try:
        qrs_amplitude = wm.get_qrs_amplitude()
        print(f"QRS Amplitude: {qrs_amplitude}")
    except:
        print("Could not extract QRS amplitude")
    
    # Extract frequency domain features if available
    try:
        print("Extracting frequency domain features...")
        freq_features = FrequencyDomainFeatures(rr_intervals, fs=fs)
        # Note: You may need to check what methods are available in FrequencyDomainFeatures
        print("Frequency domain features extracted")
    except Exception as e:
        print(f"Frequency domain features not available: {e}")
        
except Exception as e:
    print(f"Error extracting features: {e}")

    # Also analyze PPG signal
    print("\n" + "="*50)
    print("ANALYZING PPG SIGNAL")
    print("="*50)
    
    # Filter PPG signal
    sf_ppg = SignalFiltering(ppg_signal)
    filtered_ppg = sf_ppg.bandpass(lowcut=0.5, highcut=8.0, fs=fs)  # Different frequency range for PPG
    print("Applied bandpass filter (0.5-8 Hz) to PPG signal")
    
    # Extract PPG features using WaveformMorphology
    print("Extracting PPG features using WaveformMorphology...")
    wm_ppg = WaveformMorphology(filtered_ppg, fs=fs, signal_type="PPG")
    
    try:
        # Get PPG-specific features
        systolic_peaks = wm_ppg.systolic_peaks
        diastolic_peaks = wm_ppg.detect_diastolic_peak()
        dicrotic_notches = wm_ppg.detect_dicrotic_notches()
        
        print(f"Detected {len(systolic_peaks)} systolic peaks")
        print(f"Detected {len(diastolic_peaks)} diastolic peaks")
        print(f"Detected {len(dicrotic_notches)} dicrotic notches")
        
        # Calculate heart rate from PPG
        if len(systolic_peaks) > 1:
            ppg_intervals = np.diff(systolic_peaks) / fs * 1000  # Convert to milliseconds
            ppg_heart_rate = 60000 / np.mean(ppg_intervals)  # BPM
            print(f"PPG Heart rate: {ppg_heart_rate:.1f} BPM")
            
            # Extract HRV features from PPG intervals
            print("Extracting HRV features from PPG...")
            hrv_ppg = TimeDomainFeatures(ppg_intervals)
            
            ppg_hrv_results = {
                'SDNN': hrv_ppg.compute_sdnn(),
                'RMSSD': hrv_ppg.compute_rmssd(),
                'NN50': hrv_ppg.compute_nn50(),
                'pNN50': hrv_ppg.compute_pnn50(),
                'Mean_NN': hrv_ppg.compute_mean_nn(),
                'CVNN': hrv_ppg.compute_cvnn()
            }
            
            print("PPG HRV Features:")
            for feature, value in ppg_hrv_results.items():
                print(f"  {feature}: {value:.4f}")
        
    except Exception as e:
        print(f"Error extracting PPG features: {e}")

print("\n" + "="*50)
print("FEATURE EXTRACTION COMPLETED!")
print("="*50)

# Create comprehensive visualizations
print("\nCreating visualizations...")

# Create time axis for plotting
time_axis = np.arange(len(ecg_signal)) / fs

# Create subplots for ECG and PPG
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=('ECG Signal with Critical Points (Q, R, S, T, P)', 'PPG Signal with Peaks'),
    vertical_spacing=0.1
)

# Plot ECG signal
fig.add_trace(
    go.Scatter(
        x=time_axis, 
        y=filtered_signal, 
        mode='lines', 
        name='ECG Signal',
        line=dict(color='blue', width=1)
    ),
    row=1, col=1
)

# Add critical ECG points if they exist
if 'r_peaks' in locals() and len(r_peaks) > 0:
    fig.add_trace(
        go.Scatter(
            x=r_peaks/fs, 
            y=filtered_signal[r_peaks], 
            mode='markers', 
            name='R Peaks',
            marker=dict(color='red', size=8, symbol='circle'),
            text=[f'R{i+1}' for i in range(len(r_peaks))],
            textposition='top center'
        ),
        row=1, col=1
    )

if 'q_valleys' in locals() and len(q_valleys) > 0:
    fig.add_trace(
        go.Scatter(
            x=q_valleys/fs, 
            y=filtered_signal[q_valleys], 
            mode='markers', 
            name='Q Valleys',
            marker=dict(color='green', size=6, symbol='triangle-down'),
            text=[f'Q{i+1}' for i in range(len(q_valleys))],
            textposition='bottom center'
        ),
        row=1, col=1
    )

if 's_valleys' in locals() and len(s_valleys) > 0:
    fig.add_trace(
        go.Scatter(
            x=s_valleys/fs, 
            y=filtered_signal[s_valleys], 
            mode='markers', 
            name='S Valleys',
            marker=dict(color='orange', size=6, symbol='triangle-down'),
            text=[f'S{i+1}' for i in range(len(s_valleys))],
            textposition='bottom center'
        ),
        row=1, col=1
    )

if 'p_peaks' in locals() and len(p_peaks) > 0:
    fig.add_trace(
        go.Scatter(
            x=p_peaks/fs, 
            y=filtered_signal[p_peaks], 
            mode='markers', 
            name='P Peaks',
            marker=dict(color='purple', size=6, symbol='triangle-up'),
            text=[f'P{i+1}' for i in range(len(p_peaks))],
            textposition='top center'
        ),
        row=1, col=1
    )

if 't_peaks' in locals() and len(t_peaks) > 0:
    fig.add_trace(
        go.Scatter(
            x=t_peaks/fs, 
            y=filtered_signal[t_peaks], 
            mode='markers', 
            name='T Peaks',
            marker=dict(color='brown', size=6, symbol='triangle-up'),
            text=[f'T{i+1}' for i in range(len(t_peaks))],
            textposition='top center'
        ),
        row=1, col=1
    )


# Update layout
fig.update_layout(
    height=800,
    title_text="VitalDSP Signal Analysis: ECG with Critical Points",
    title_x=0.5,
    showlegend=True
)

# Update x and y axis labels
fig.update_xaxes(title_text="Time (seconds)", row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=1)

# Show the plot
fig.show()

# Create a summary statistics plot
print("\nCreating summary statistics visualization...")

# Prepare data for summary plot
if 'hrv_results' in locals():
    hrv_names = list(hrv_results.keys())
    hrv_values = list(hrv_results.values())
    
    # Create bar chart for HRV features
    fig_hrv = go.Figure(data=[
        go.Bar(
            x=hrv_names,
            y=hrv_values,
            marker_color='lightblue',
            text=[f'{v:.3f}' for v in hrv_values],
            textposition='auto'
        )
    ])
    
    fig_hrv.update_layout(
        title="Heart Rate Variability (HRV) Features",
        xaxis_title="HRV Metrics",
        yaxis_title="Values",
        height=500
    )
    
    fig_hrv.show()

print("\n" + "="*60)
print("VISUALIZATION COMPLETED!")
print("="*60)
print("Summary:")
print(f"- ECG Signal: {len(ecg_signal)} samples at {fs} Hz")
if 'r_peaks' in locals():
    print(f"- Detected {len(r_peaks)} R-peaks")
    print(f"- Heart Rate: {heart_rate:.1f} BPM")
if 'ppg_signal' in locals() and len(ppg_signal) > 1:
    print(f"- PPG Signal: {len(ppg_signal)} samples at {fs} Hz")
    if 'systolic_peaks' in locals():
        print(f"- Detected {len(systolic_peaks)} systolic peaks")
print("="*60)

In [ ]:
import numpy as np
import vitalDSP
from vitalDSP.utils.synthesize_data import generate_ecg_signal, generate_synthetic_ppg
from vitalDSP.filtering.signal_filtering import SignalFiltering
from vitalDSP.physiological_features.waveform import WaveformMorphology
from vitalDSP.physiological_features.time_domain import TimeDomainFeatures
from vitalDSP.physiological_features.frequency_domain import FrequencyDomainFeatures
from vitalDSP.physiological_features.hrv_analysis import HRVFeatures
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from vitalDSP.filtering.artifact_removal import ArtifactRemoval

# Generate synthetic ECG signal using VitalDSP
fs = 128  # Sampling frequency
duration = 10  # seconds
hr_mean = 75  # beats per minute
noise_amplitude = 0.1

print("Generating synthetic ECG signal using VitalDSP...")
ecg_signal = generate_ecg_signal(
    sfecg=fs, 
    N=duration, 
    Anoise=noise_amplitude, 
    hrmean=hr_mean
)

print(f"Generated ECG signal: {len(ecg_signal)} samples at {fs} Hz")

# Generate synthetic PPG signal for comparison
print("\nGenerating synthetic PPG signal using VitalDSP...")
ppg_time, ppg_signal = generate_synthetic_ppg(
    duration=duration,
    sampling_rate=fs,
    heart_rate=hr_mean,
    noise_level=noise_amplitude
)

print(f"Generated PPG signal: {len(ppg_signal)} samples at {fs} Hz")

# Apply filtering to both signals
print("\nApplying signal filtering...")

# ECG filtering
sf_ecg = SignalFiltering(ecg_signal)
filtered_ecg = sf_ecg.bandpass(lowcut=0.3, highcut=20.0, fs=fs)
print("Applied bandpass filter (0.3-20 Hz) to ECG signal")

# PPG filtering
sf_ppg = SignalFiltering(ppg_signal)
filtered_ppg = sf_ppg.bandpass(lowcut=0.5, highcut=8.0, fs=fs)
print("Applied bandpass filter (0.5-8 Hz) to PPG signal")

# Artifact removal
ar = ArtifactRemoval(ecg_signal)
clean_ecg = ar.median_filter_removal(kernel_size=3)
print("Applied median filter artifact removal to ECG signal")

# Create comprehensive visualization comparing original vs filtered signals
print("\nCreating signal comparison visualizations...")

# Create time axes
ecg_time = np.arange(len(ecg_signal)) / fs
ppg_time_axis = np.arange(len(ppg_signal)) / fs

# Create subplots for signal comparison
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'ECG Signal: Original vs Filtered', 
        'PPG Signal: Original vs Filtered',
        'ECG Signal: Original vs Artifact Removed',
        'Signal Quality Comparison'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Plot 1: ECG Original vs Filtered
fig.add_trace(
    go.Scatter(
        x=ecg_time, 
        y=ecg_signal, 
        mode='lines', 
        name='ECG Original',
        line=dict(color='blue', width=1, dash='solid'),
        opacity=0.7
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=ecg_time, 
        y=filtered_ecg, 
        mode='lines', 
        name='ECG Filtered',
        line=dict(color='red', width=1.5, dash='solid')
    ),
    row=1, col=1
)

# Plot 2: PPG Original vs Filtered
fig.add_trace(
    go.Scatter(
        x=ppg_time_axis, 
        y=ppg_signal, 
        mode='lines', 
        name='PPG Original',
        line=dict(color='green', width=1, dash='solid'),
        opacity=0.7
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=ppg_time_axis, 
        y=filtered_ppg, 
        mode='lines', 
        name='PPG Filtered',
        line=dict(color='orange', width=1.5, dash='solid')
    ),
    row=1, col=2
)

# Plot 3: ECG Original vs Artifact Removed
fig.add_trace(
    go.Scatter(
        x=ecg_time, 
        y=ecg_signal, 
        mode='lines', 
        name='ECG Original',
        line=dict(color='blue', width=1, dash='solid'),
        opacity=0.7,
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=ecg_time, 
        y=clean_ecg, 
        mode='lines', 
        name='ECG Artifact Removed',
        line=dict(color='purple', width=1.5, dash='solid')
    ),
    row=2, col=1
)

# Plot 4: Signal Quality Metrics Comparison
# Calculate signal quality metrics
ecg_snr_original = 20 * np.log10(np.std(ecg_signal) / (np.std(ecg_signal - np.mean(ecg_signal)) + 1e-10))
ecg_snr_filtered = 20 * np.log10(np.std(filtered_ecg) / (np.std(filtered_ecg - np.mean(filtered_ecg)) + 1e-10))
ppg_snr_original = 20 * np.log10(np.std(ppg_signal) / (np.std(ppg_signal - np.mean(ppg_signal)) + 1e-10))
ppg_snr_filtered = 20 * np.log10(np.std(filtered_ppg) / (np.std(filtered_ppg - np.mean(filtered_ppg)) + 1e-10))

# Calculate RMS values
ecg_rms_original = np.sqrt(np.mean(ecg_signal**2))
ecg_rms_filtered = np.sqrt(np.mean(filtered_ecg**2))
ppg_rms_original = np.sqrt(np.mean(ppg_signal**2))
ppg_rms_filtered = np.sqrt(np.mean(filtered_ppg**2))

# Create bar chart for quality metrics
quality_metrics = ['ECG SNR', 'PPG SNR', 'ECG RMS', 'PPG RMS']
original_values = [ecg_snr_original, ppg_snr_original, ecg_rms_original, ppg_rms_original]
filtered_values = [ecg_snr_filtered, ppg_snr_filtered, ecg_rms_filtered, ppg_rms_filtered]

fig.add_trace(
    go.Bar(
        x=quality_metrics,
        y=original_values,
        name='Original',
        marker_color='lightblue',
        opacity=0.7
    ),
    row=2, col=2
)

fig.add_trace(
    go.Bar(
        x=quality_metrics,
        y=filtered_values,
        name='Filtered',
        marker_color='lightcoral',
        opacity=0.7
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="VitalDSP Signal Processing: Original vs Filtered Signals",
    title_x=0.5,
    showlegend=True
)

# Update axis labels
fig.update_xaxes(title_text="Time (seconds)", row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=1)
fig.update_xaxes(title_text="Time (seconds)", row=1, col=2)
fig.update_yaxes(title_text="Amplitude", row=1, col=2)
fig.update_xaxes(title_text="Time (seconds)", row=2, col=1)
fig.update_yaxes(title_text="Amplitude", row=2, col=1)
fig.update_xaxes(title_text="Quality Metrics", row=2, col=2)
fig.update_yaxes(title_text="Values", row=2, col=2)

# Show the plot
fig.show()

# Print signal quality summary
print("\n" + "="*60)
print("SIGNAL QUALITY ANALYSIS")
print("="*60)
print(f"ECG Signal Quality:")
print(f"  Original SNR: {ecg_snr_original:.2f} dB")
print(f"  Filtered SNR: {ecg_snr_filtered:.2f} dB")
print(f"  SNR Improvement: {ecg_snr_filtered - ecg_snr_original:.2f} dB")
print(f"  Original RMS: {ecg_rms_original:.4f}")
print(f"  Filtered RMS: {ecg_rms_filtered:.4f}")

print(f"\nPPG Signal Quality:")
print(f"  Original SNR: {ppg_snr_original:.2f} dB")
print(f"  Filtered SNR: {ppg_snr_filtered:.2f} dB")
print(f"  SNR Improvement: {ppg_snr_filtered - ppg_snr_original:.2f} dB")
print(f"  Original RMS: {ppg_rms_original:.4f}")
print(f"  Filtered RMS: {ppg_rms_filtered:.4f}")

print(f"\nArtifact Removal:")
print(f"  Original ECG RMS: {ecg_rms_original:.4f}")
print(f"  Clean ECG RMS: {np.sqrt(np.mean(clean_ecg**2)):.4f}")
print(f"  Noise Reduction: {((ecg_rms_original - np.sqrt(np.mean(clean_ecg**2))) / ecg_rms_original * 100):.1f}%")
print("="*60)


In [ ]:
# Step 1: Import the feature extraction classes
from vitalDSP.feature_engineering.morphology_features import PhysiologicalFeatureExtractor
from vitalDSP.physiological_features.hrv_analysis import HRVFeatures


# Step 2: Extract physiological features from ECG signal
print("1. Extracting physiological features...")
extractor = PhysiologicalFeatureExtractor(filtered_ecg, fs=fs)
features = extractor.extract_features(signal_type="ECG")

print(f"   ✓ Extracted {len(features) if features else 0} features")

# Step 3: Show some example features
if features:
    print("2. Example features:")
    for key, value in features.items():
            print(f"   • {key}: {value}")

    

# Step 4: Extract HRV features
print("3. Extracting HRV features...")
hrv = HRVFeatures(signals=filtered_ecg, signal_type="ECG")
hrv_features = hrv.compute_all_features()
    
print(f"   ✓ Extracted {len(hrv_features) if hrv_features else 0} HRV features")
    
if hrv_features:
    print("   Example HRV features:")

    for key, value in hrv_features.items():
        print(f"   • {key}: {value}")


In [ ]:
from vitalDSP.respiratory_analysis.respiratory_analysis import RespiratoryAnalysis


sf_ecg = SignalFiltering(ecg_signal)
resp_low = 0.05
resp_high = 0.4
filtered_ecg_resp = sf_ecg.bandpass(lowcut=resp_low, highcut=resp_high, fs=fs)
print(f"Filtered ECG signal: {len(filtered_ecg_resp)} samples at {60*resp_low}-{60*resp_high} bpm")

resp_analysis = RespiratoryAnalysis(filtered_ecg_resp, fs)
respiratory_rate = resp_analysis.compute_respiratory_rate(
    method="peaks",
    correction_method=None,
    min_breath_duration=1.5,
    max_breath_duration=5,
    preprocess_config=None
)
print(respiratory_rate)


In [ ]:
%matplotlib inline
from vitalDSP.advanced_computation.emd import EMD
from vitalDSP.advanced_computation.pitch_shift import PitchShift
from vitalDSP.advanced_computation.anomaly_detection import AnomalyDetection
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style
sns.set_style("whitegrid")
sns.set_palette("husl")

# Perform EMD decomposition
emd = EMD(ecg_signal)
imfs = emd.emd()
print(f"Number of IMFs: {len(imfs)}")

# Simple plot of all IMFs
plt.figure(figsize=(12, 8))

# Plot original signal
plt.subplot(len(imfs) + 1, 1, 1)
plt.plot(ecg_signal, 'k-', linewidth=1.5)
plt.title('Original ECG Signal', fontweight='bold')
plt.ylabel('Amplitude')
plt.grid(True, alpha=0.3)

# Plot each IMF
for i, imf in enumerate(imfs):
    plt.subplot(len(imfs) + 1, 1, i + 2)
    plt.plot(imf, linewidth=1.2, color=sns.color_palette("husl", len(imfs))[i])
    plt.title(f'IMF {i + 1}', fontweight='bold')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Simple heatmap
plt.figure(figsize=(12, 6))
imf_matrix = np.array(imfs)
sns.heatmap(imf_matrix, 
            yticklabels=[f'IMF {i+1}' for i in range(len(imfs))],
            xticklabels=False,
            cmap='RdBu_r',
            center=0)
plt.title('IMFs Heatmap', fontweight='bold')
plt.ylabel('Intrinsic Mode Functions')
plt.tight_layout()
plt.show()

# Plot original signal
plt.subplot(len(imfs) + 1, 1, 1)
plt.plot(ecg_signal, 'k-', linewidth=1.5)
plt.title('Original ECG Signal', fontweight='bold')
plt.ylabel('Amplitude')
plt.grid(True, alpha=0.3)

# Plot each IMF
for i, imf in enumerate(imfs):
    plt.subplot(len(imfs) + 1, 1, i + 2)
    plt.plot(imf, linewidth=1.2, color=sns.color_palette("husl", len(imfs))[i])
    plt.title(f'IMF {i + 1}', fontweight='bold')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Simple heatmap
plt.figure(figsize=(12, 6))
imf_matrix = np.array(imfs)
sns.heatmap(imf_matrix, 
            yticklabels=[f'IMF {i+1}' for i in range(len(imfs))],
            xticklabels=False,
            cmap='RdBu_r',
            center=0)
plt.title('IMFs Heatmap', fontweight='bold')
plt.ylabel('Intrinsic Mode Functions')
plt.tight_layout()
plt.show()



In [ ]:
# Pitch Shift filtering
pitch_shift_filter = PitchShift(ecg_signal,sampling_rate=fs)
filtered_signal = pitch_shift_filter.shift_pitch(semitones=2)


fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(filtered_signal)), y=ecg_signal, mode='lines', name='Original Signal'))
fig.add_trace(go.Scatter(x=np.arange(len(filtered_signal)), y=filtered_signal, mode='lines', name='Pitch Shifted Signal'))
fig.show()

# Anomaly detection
anomaly_detector = AnomalyDetection(ecg_signal)
anomalies = anomaly_detector.detect_anomalies(method="z_score")

fig = go.Figure()
fig.add_trace(go.Scatter(y=np.arange(len(ecg_signal)), x=anomalies, mode='lines', name='Anomlies Lines'))
fig.show()


In [ ]:
from vitalDSP.preprocess.preprocess_operations import PreprocessConfig
from vitalDSP.preprocess.preprocess_operations import PreprocessConfig, preprocess_signal

# Configure preprocessing
config = PreprocessConfig(
    filter_type="bandpass",
    noise_reduction_method="wavelet",
    lowcut=0.1,
    highcut=4.5,
    order=4,
    wavelet_name="haar",
    level=1,
    respiratory_mode=False
)

# Apply preprocessing
clean_signal = preprocess_signal(
    signal=ecg_signal,
    sampling_rate=fs,
    filter_type=config.filter_type,
    lowcut=config.lowcut,
    highcut=config.highcut,
    order=config.order,
    noise_reduction_method=config.noise_reduction_method,
    wavelet_name=config.wavelet_name,
    level=config.level,
    window_length=config.window_length,
    polyorder=config.polyorder,
    kernel_size=config.kernel_size,
    sigma=config.sigma,
    respiratory_mode=config.respiratory_mode,
    repreprocess=config.repreprocess
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_signal)), y=ecg_signal, mode='lines', name='Original Signal'))
fig.add_trace(go.Scatter(x=np.arange(len(clean_signal)), y=clean_signal, mode='lines', name='Preprocessed Signal'))
fig.show()

In [ ]:
# 1. Preprocessing
from vitalDSP.preprocess.preprocess_operations import PreprocessConfig, preprocess_signal

# Configure preprocessing
config = PreprocessConfig(
    filter_type="bandpass",
    noise_reduction_method="wavelet",
    lowcut=0.1,
    highcut=4.5,
    order=4
)

# Apply preprocessing
clean_signal = preprocess_signal(
    signal=ecg_signal,
    sampling_rate=fs,
    filter_type=config.filter_type,
    lowcut=config.lowcut,
    highcut=config.highcut,
    order=config.order,
    noise_reduction_method=config.noise_reduction_method
)

print("✅ Preprocessing completed!")

# 2. Feature extraction
from vitalDSP.feature_engineering.morphology_features import PhysiologicalFeatureExtractor

extractor = PhysiologicalFeatureExtractor(clean_signal, fs=fs)
features = extractor.extract_features(signal_type="ECG")

print("✅ Features extracted!")
print(f"Features: {features}")

# 3. Quality assessment
from vitalDSP.signal_quality_assessment.signal_quality import SignalQuality

sq = SignalQuality(ecg_signal, clean_signal)
quality_metrics = {
    'snr': sq.snr(),
    'psnr': sq.psnr(),
    'mse': sq.mse()
}

print("✅ Quality assessment completed!")
print(f"Quality metrics: {quality_metrics}")